In [90]:
import pandas as pd
import sqlite3

# Database file (Creates if it doesn't exist)
db_name = "stocks_data.db"

# Connect to SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [91]:
from yahooquery import Ticker

In [84]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [92]:
df_NSE = pd.read_csv('/Users/jayaprakash/Downloads/NSELIST.csv')


In [31]:
#this code to process if there is any new ticker in the input list
df_NSE = pd.read_csv('/Users/jayaprakash/Downloads/NSELIST.csv')
df_NSE1 = df_NSE['TICKER']
df_NSE18 = df_NSE1.tolist()
# df_NSE
df_NSE18

df_st_table= pd.read_sql("SELECT distinct ticker FROM stock_prices", conn)
st_list = df_st_table['ticker'].tolist()
missed_tickers =  []
for i in df_NSE18:
    if i not in st_list:
        missed_tickers.append(i)

missed_tickers

subset = missed_tickers[:100]  # Select first 300 elements
subset

dailyprice = []
import time

for i in subset:
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period='max', interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

if dailyprice:
    df_daily_values = pd.concat(dailyprice)
else:
    print("No ticker to process")


No ticker to process


In [110]:
df_daily_values2 = pd.concat(dailyprice)
df_daily_values2.reset_index(inplace=True)
df_daily_values3 = df_daily_values2.drop(columns=['dividends','splits'], errors='ignore')
df_daily_values3
#1248024

/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_13456/2945664598.py:1: RuntimeWarning: can't compare datetime.datetime to datetime.date, sort order is undefined for incomparable objects.
  df_daily_values2 = pd.concat(dailyprice)


,symbol,date,open,high,low,close,volume,adjclose,ticker
0,SBICARD.NS,2020-03-16,661.000000,755.000000,656.000000,681.400024,60887005.0,671.316589,SBICARD.NS
1,SBICARD.NS,2020-03-17,691.000000,747.950012,689.000000,730.900024,15145135.0,720.084045,SBICARD.NS
2,SBICARD.NS,2020-03-18,752.099976,769.000000,680.000000,689.200012,8949832.0,679.001160,SBICARD.NS
3,SBICARD.NS,2020-03-19,660.000000,717.000000,650.000000,692.049988,6432046.0,681.808899,SBICARD.NS
4,SBICARD.NS,2020-03-20,706.250000,730.000000,690.250000,724.500000,6492226.0,713.778748,SBICARD.NS
...,...,...,...,...,...,...,...,...,...
494944,ZYDUSWELL.NS,2025-03-20,1624.949951,1667.150024,1624.949951,1650.449951,71909.0,1650.449951,ZYDUSWELL.NS
494945,ZYDUSWELL.NS,2025-03-21,1661.550049,1676.949951,1645.349976,1668.800049,16485.0,1668.800049,ZYDUSWELL.NS
494946,ZYDUSWELL.NS,2025-03-24,1668.800049,1672.949951,1649.599976,1667.599976,23153.0,1667.599976,ZYDUSWELL.NS
494947,ZYDUSWELL.NS,2025-03-25,1675.000000,1688.750000,1651.000000,1680.150024,33812.0,1680.150024,ZYDUSWELL.NS


In [85]:
#this is identify number of days stock prices to be pulled from the last processed date
df_st_table= pd.read_sql("SELECT ticker, max(date(date)) as date FROM stock_prices group by ticker", conn)
df_st_table['date'] = pd.to_datetime(df_st_table['date'])#.dt.date
# Calculate days from today (change 'today' as needed)
df_st_table['Days_From_Today'] = (pd.Timestamp.today().normalize() - df_st_table['date']).dt.days.astype('str') + str('d')
#df_st_table['Days_From_Today'] = df_st_table['date'].apply(
#     lambda x: str((pd.Timestamp.today().normalize() - x).days) + 'd'
# )
# df_NSE_st = df_NSE1.merge(df_st_table, left_on='TICKER', right_on='ticker', how='left')
# df_NSE_st = df_NSE_st.assign(Days_From_Today=df_NSE_st['Days_From_Today'].fillna("max"))
# df_NSE_st_filtered = df_NSE_st[df_NSE_st['Days_From_Today'] == 'max']
# df_NSE_st
#df_st_table['date'].max()
df_st_table

,ticker,date,Days_From_Today
0,20MICRONS.NS,2025-03-21,12d
1,21STCENMGM.NS,2025-03-21,12d
2,360ONE.NS,2025-03-21,12d
3,3IINFOLTD.NS,2025-03-21,12d
4,3MINDIA.NS,2025-03-21,12d
...,...,...,...
2085,ZOTA.NS,2025-03-26,7d
2086,ZUARI.NS,2025-03-26,7d
2087,ZUARIIND.NS,2025-03-26,7d
2088,ZYDUSLIFE.NS,2025-03-26,7d


In [86]:
dailyprice = []
import time

for index, row in df_st_table.iterrows():
    i = row['ticker']
    k = row['Days_From_Today']
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period=k, interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

df_daily_values = pd.concat(dailyprice)

error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}


/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_6443/571964099.py:17: RuntimeWarning: can't compare datetime.datetime to datetime.date, sort order is undefined for incomparable objects.
  df_daily_values = pd.concat(dailyprice)


In [88]:
df_daily_values2 = pd.concat(dailyprice)
df_daily_values2.reset_index(inplace=True)
df_daily_values3 = df_daily_values2.drop(columns=['dividends','splits'], errors='ignore')
df_daily_values3

/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_6443/1155183703.py:1: RuntimeWarning: can't compare datetime.datetime to datetime.date, sort order is undefined for incomparable objects.
  df_daily_values2 = pd.concat(dailyprice)


,symbol,date,open,high,low,close,volume,adjclose,ticker
0,20MICRONS.NS,2025-03-17,194.000000,195.270004,186.100006,187.179993,65883.0,187.179993,20MICRONS.NS
1,20MICRONS.NS,2025-03-18,187.179993,187.179993,187.179993,187.179993,0.0,187.179993,20MICRONS.NS
2,20MICRONS.NS,2025-03-19,199.000000,214.000000,198.000000,201.149994,147127.0,201.149994,20MICRONS.NS
3,20MICRONS.NS,2025-03-20,202.000000,204.949997,200.419998,201.110001,75022.0,201.110001,20MICRONS.NS
4,20MICRONS.NS,2025-03-21,202.649994,205.710007,201.490005,204.179993,68420.0,204.179993,20MICRONS.NS
...,...,...,...,...,...,...,...,...,...
18045,ZYDUSWELL.NS,2025-03-26,1674.849976,1674.849976,1612.500000,1618.050049,19633.0,1618.050049,ZYDUSWELL.NS
18046,ZYDUSWELL.NS,2025-03-27,1618.000000,1705.000000,1603.300049,1690.599976,49753.0,1690.599976,ZYDUSWELL.NS
18047,ZYDUSWELL.NS,2025-03-28,1676.000000,1729.000000,1666.000000,1680.500000,27725.0,1680.500000,ZYDUSWELL.NS
18048,ZYDUSWELL.NS,2025-04-01,1680.500000,1701.949951,1650.000000,1678.300049,11376.0,1678.300049,ZYDUSWELL.NS


In [89]:
for _, row in df_daily_values3.iterrows():
    try:
        cursor.execute("""
            INSERT INTO stock_prices (symbol, date, open, high, low, close, volume, adjclose, ticker)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(ticker, date) DO UPDATE SET
                open = excluded.open,
                high = excluded.high,
                low = excluded.low,
                close = excluded.close,
                volume = excluded.volume,
                adjclose = excluded.adjclose;
        """, tuple(row))
    except Exception as e:
        print(f"Error inserting data: {e}")

/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_6443/4254892763.py:3: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""
/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_6443/4254892763.py:3: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


In [78]:
from yahooquery import Ticker


tickers = Ticker(symbols)

# Retrieve each company's profile information
data = tickers.asset_profile


In [135]:
# df_NSE_comp = df_NSE[['COMPANY','TICKER']]
# df_NSE_comp = df_NSE_comp.head(1)

# final_df = pd.DataFrame()
# for index, row in df_NSE_comp.iterrows():
#     comp =  row['COMPANY']
#     tk = row['TICKER']
#     tickers = Ticker(tk)
#     data = tickers.asset_profile
#     data['company_name'] = comp
#     df_company = pd.DataFrame.from_dict(data, orient="index").T
#     final_df = pd.concat([final_df,df_company])
# final_df






In [131]:
import pandas as pd
from yahooquery import Ticker  # Library to fetch company details

# Step 1: Select only relevant columns from df_NSE (First row for testing)
df_NSE_comp = df_NSE[['COMPANY', 'TICKER']] 

# Step 2: Create an empty DataFrame to store final results
final_df = pd.DataFrame()

# Step 3: Define only the fields we want from the fetched company data
required_fields = ['country', 'website', 'maxAge',
                   'industry', 'sector', 'longBusinessSummary', 'fullTimeEmployees']

# Step 4: Loop through each row (company) in the DataFrame
for index, row in df_NSE_comp.iterrows():
    comp = row['COMPANY']  # Get the company name
    tk = row['TICKER']  # Get the stock ticker symbol

    print(f"Fetching data for: {comp} ({tk})")  # Debugging print

    tickers = Ticker(tk)  # Fetch company details
    data = tickers.asset_profile  # This is a dictionary of company details

    # Step 5: Ensure the API returned valid data
    if isinstance(data, dict) and tk in data and isinstance(data[tk], dict):
        company_data = data[tk]  # Extract only the company details for this ticker

        # Step 6: Extract only required fields, ignore others
        extracted_data = {field: company_data.get(field, None) for field in required_fields}

        # Step 7: Add company name manually (not present in API data)
        extracted_data['company_name'] = comp  
        extracted_data['ticker'] = tk  


        # Step 8: Convert dictionary to a single-row DataFrame
        df_company = pd.DataFrame([extracted_data])

        # Step 9: Append the new DataFrame to our final DataFrame
        final_df = pd.concat([final_df, df_company], ignore_index=True)
    else:
        print(f"Error: Unexpected data format for {tk}: {data}")  # Debugging message

# Step 10: Display the structured DataFrame


Fetching data for: 20 Microns Limited (20MICRONS.NS)
Fetching data for: 21st Century Management Services Limited (21STCENMGM.NS)
Fetching data for: 360 ONE WAM LIMITED (360ONE.NS)
Fetching data for: 3i Infotech Limited (3IINFOLTD.NS)
Fetching data for: 3M India Limited (3MINDIA.NS)
Fetching data for: 3P Land Holdings Limited (3PLAND.NS)
Error: Unexpected data format for 3PLAND.NS: {'3PLAND.NS': 'No fundamentals data found for symbol: 3PLAND.NS'}
Fetching data for: 5Paisa Capital Limited (5PAISA.NS)
Fetching data for: 63 moons technologies limited (63MOONS.NS)
Fetching data for: A2Z Infra Engineering Limited (A2ZINFRA.NS)
Fetching data for: AAA Technologies Limited (AAATECH.NS)
Fetching data for: Aadhar Housing Finance Limited (AADHARHFC.NS)
Fetching data for: Aakash Exploration Services Limited (AAKASH.NS)
Fetching data for: Aarey Drugs & Pharmaceuticals Limited (AAREYDRUGS.NS)
Fetching data for: Aaron Industries Limited (AARON.NS)
Fetching data for: Aartech Solonics Limited (AARTECH.N

In [120]:
final_df.dtypes

country                object
website                object
maxAge                  int64
industry               object
sector                 object
longBusinessSummary    object
fullTimeEmployees       int64
company_name           object
ticker                 object
dtype: object

In [133]:
import sqlite3
import pandas as pd

# Function to insert or update company data
def insert_company_info(df):
    db_name = "stocks_data.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    insert_query = """
    INSERT INTO company_info (
        country, website, maxAge, industry, sector, longBusinessSummary, 
        fullTimeEmployees, company_name, ticker
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ON CONFLICT(ticker) DO UPDATE SET 
        country = excluded.country,
        website = excluded.website,
        maxAge = excluded.maxAge,
        industry = excluded.industry,
        sector = excluded.sector,
        longBusinessSummary = excluded.longBusinessSummary,
        fullTimeEmployees = excluded.fullTimeEmployees,
        company_name = excluded.company_name;
    """

    # Check if the required columns exist
    expected_columns = [
        "country", "website", "maxAge", "industry", "sector",
        "longBusinessSummary", "fullTimeEmployees", "company_name", "ticker"
    ]

    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Missing columns in DataFrame: {missing_columns}")

    # Insert Data
    for _, row in df.iterrows():
        values = (
            row.get("country", None),
            row.get("website", None),
            row.get("maxAge", None),
            row.get("industry", None),
            row.get("sector", None),
            row.get("longBusinessSummary", None),
            row.get("fullTimeEmployees", None),
            row.get("company_name", None),
            row.get("ticker", None),
        )
        cursor.execute(insert_query, values)

    conn.commit()
    cursor.close()
    conn.close()

    print(f"Inserted/Updated {len(df)} records successfully!")

# Example: Check DataFrame columns before inserting
print("Columns in final_df:", final_df.columns)

# Insert data
insert_company_info(final_df)


Columns in final_df: Index(['country', 'website', 'maxAge', 'industry', 'sector',
       'longBusinessSummary', 'fullTimeEmployees', 'company_name', 'ticker'],
      dtype='object')
Inserted/Updated 2080 records successfully!


In [137]:
#pd.read_sql('''select * from company_info''', conn)

,country,website,maxAge,industry,sector,longBusinessSummary,fullTimeEmployees,company_name,ticker
0,India,https://www.20microns.com,86400,Other Industrial Metals & Mining,Basic Materials,20 Microns Limited manufactures and markets mi...,384.0,20 Microns Limited,20MICRONS.NS
1,India,https://www.tcms.bz,86400,Capital Markets,Financial Services,Twentyfirst Century Management Services Limite...,10.0,21st Century Management Services Limited,21STCENMGM.NS
2,India,https://www.360.one,86400,Asset Management,Financial Services,360 One Wam Limited engages in the provision o...,1200.0,360 ONE WAM LIMITED,360ONE.NS
3,India,https://www.3i-infotech.com,86400,Information Technology Services,Technology,3i Infotech Limited provides IP based software...,5600.0,3i Infotech Limited,3IINFOLTD.NS
4,India,https://www.3mindia.in,86400,Conglomerates,Industrials,3M India Limited manufactures and trades in va...,1033.0,3M India Limited,3MINDIA.NS
...,...,...,...,...,...,...,...,...,...
2075,India,https://www.zotahealthcare.com,86400,Biotechnology,Healthcare,"Zota Health Care Limited develops, manufacture...",445.0,Zota Health Care LImited,ZOTA.NS
2076,India,https://www.zuari.in,86400,Agricultural Inputs,Basic Materials,Zuari Agro Chemicals Limited engages in manufa...,NaN,Zuari Agro Chemicals Limited,ZUARI.NS
2077,India,https://www.adventz.com,86400,Conglomerates,Industrials,Zuari Industries Limited engages in agricultur...,329.0,ZUARI INDUSTRIES LIMITED,ZUARIIND.NS
2078,India,https://zyduslife.com,86400,Drug Manufacturers - Specialty & Generic,Healthcare,Zydus Lifesciences Limited engages in the rese...,26921.0,Zydus Lifesciences Limited,ZYDUSLIFE.NS
